In [29]:
import numpy as np
import pandas as pd
import os
import glob

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [30]:
df_test_pitch = pd.read_csv('../yakiu_rawdata/test_pitch.csv')
df_test_player = pd.read_csv('../yakiu_rawdata/test_player.csv')
df_train_pitch = pd.read_csv('../yakiu_rawdata/train_pitch.csv')
df_train_player = pd.read_csv('../yakiu_rawdata/train_player.csv')
df_sample = pd.read_csv('../yakiu_rawdata/sample_submit_ball_type.csv')

# プレイデータの加工

In [56]:
df_test_pitch

,データ内連番,年度,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,一塁手ID,二塁手ID,三塁手ID,遊撃手ID,左翼手ID,中堅手ID,右翼手ID,成績対象投手ID,成績対象打者ID
0,0,2018,2018033001,1,1,2018-03-30,18:03:00,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1200041,1,右,先発,1,1,1,1,1500086,5,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,1500086
1,1,2018,2018033001,2,2,2018-03-30,18:03:39,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1200041,1,右,先発,1,2,2,2,400001,5,左,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400001
2,2,2018,2018033001,3,3,2018-03-30,18:03:57,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1200041,1,右,先発,1,2,3,3,400001,5,左,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400001
3,3,2018,2018033001,4,4,2018-03-30,18:04:40,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,3,1,1200041,1,右,先発,1,3,4,4,400057,5,左,3,右,1,0,0,2,0,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400057
4,4,2018,2018033001,5,5,2018-03-30,18:04:56,1,5,1,東京ドーム,セ・リーグ公式戦,1,表,3,2,1200041,1,右,先発,1,3,5,5,400057,5,左,3,右,1,0,0,2,1,0,___,NaN,NaN,NaN,1300027,1400101,1600059,1200136,700003,1600184,600037,900397,1200041,400057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521645,521645,2019,2019093001,289,285,2019-09-30,21:10:05,5,4,5,甲子園,セ・リーグ公式戦,9,表,3,8,11807,5,右,救援,6,3,17,17,700034,4,右,4,一,2,3,0,1,1,2,_2_,NaN,1700103.0,NaN,1300073,1300072,1400172,1600113,400001,11771,1800065,1500086,11807,700034
521646,521646,2019,2019093001,290,286,2019-09-30,21:10:53,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,1,11807,5,右,救援,6,4,18,18,1100069,4,左,5,三,4,3,0,2,0,0,_2_,NaN,1700103.0,NaN,1300073,1300072,1400172,1600113,400001,11771,1800065,1500086,11807,1100069
521647,521647,2019,2019093001,291,287,2019-09-30,21:11:21,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,2,11807,5,右,救援,6,4,19,19,1100069,4,左,5,三,4,3,0,2,0,1,_2_,NaN,1700103.0,NaN,1300073,1300072,1400172,1600113,400001,11771,1800065,1500086,11807,1100069
521648,521648,2019,2019093001,292,288,2019-09-30,21:12:05,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,3,11807,5,右,救援,6,4,20,20,1100069,4,左,5,三,4,3,0,2,0,2,_2_,NaN,1700103.0,NaN,1300073,1300072,1400172,1600113,400001,11771,1800065,1500086,11807,1100069


In [32]:
df_train_pitch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257117 entries, 0 to 257116
Data columns (total 51 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   データ内連番          257117 non-null  int64  
 1   球種              257117 non-null  int64  
 2   投球位置区域          257117 non-null  int64  
 3   年度              257117 non-null  int64  
 4   試合ID            257117 non-null  int64  
 5   試合内連番           257117 non-null  int64  
 6   試合内投球数          257117 non-null  int64  
 7   日付              257117 non-null  object 
 8   時刻              257117 non-null  object 
 9   ホームチームID        257117 non-null  int64  
 10  アウェイチームID       257117 non-null  int64  
 11  球場ID            257117 non-null  int64  
 12  球場名             257117 non-null  object 
 13  試合種別詳細          257117 non-null  object 
 14  イニング            257117 non-null  int64  
 15  表裏              257117 non-null  object 
 16  イニング内打席数        257117 non-null  int64  
 17  打席内投球数    

In [33]:
df_train_pitch2 =  df_train_pitch.drop(['年度','一塁手ID','二塁手ID','三塁手ID','遊撃手ID','左翼手ID','中堅手ID','左翼手ID'], axis=1)

In [34]:
df_train_pitch2.head(5)

,データ内連番,球種,投球位置区域,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,右翼手ID,成績対象投手ID,成績対象打者ID
0,0,0,0,2017033101,1,1,2017-03-31,18:03:00,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1500001,1,右,先発,1,1,1,1,900410,4,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,900410
1,1,0,8,2017033101,2,2,2017-03-31,18:03:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,2,1500001,1,右,先発,1,1,2,2,900410,4,左,1,中,1,0,0,0,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,900410
2,2,0,5,2017033101,3,3,2017-03-31,18:03:23,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,3,1500001,1,右,先発,1,1,3,3,900410,4,左,1,中,1,0,0,0,0,2,___,NaN,NaN,NaN,1300027,900397,1500001,900410
3,3,0,12,2017033101,4,4,2017-03-31,18:04:01,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1500001,1,右,先発,1,2,4,4,11436,4,右,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,11436
4,4,0,8,2017033101,5,5,2017-03-31,18:04:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1500001,1,右,先発,1,2,5,5,11436,4,右,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,11436


In [35]:
df_train_pitch2['count'] = df_train_pitch2[['プレイ前ストライク数', 'プレイ前ボール数', 'プレイ前アウト数']].apply(lambda x: '{}-{}-{}'.format(x[0], x[1], x[2]), axis=1)

In [36]:
df_train_pitch2['count']

0         0-0-0
1         1-0-0
2         2-0-0
3         0-0-1
4         1-0-1
          ...  
257112    0-0-2
257113    1-0-2
257114    1-1-2
257115    2-1-2
257116    2-2-2
Name: count, Length: 257117, dtype: object

In [37]:
df_train_pitch2_h = df_train_pitch2.query('表裏=="表"')
df_train_pitch2_h['pitch_team_score'] = df_train_pitch2_h['プレイ前アウェイチーム得点数']-df_train_pitch2_h['プレイ前ホームチーム得点数']
df_train_pitch2_a = df_train_pitch2.query('表裏=="裏"')
df_train_pitch2_a['pitch_team_score'] = df_train_pitch2_a['プレイ前ホームチーム得点数']-df_train_pitch2_a['プレイ前アウェイチーム得点数']
df_train_pitch3 = pd.concat([df_train_pitch2_h,df_train_pitch2_a])
df_train_pitch3 =df_train_pitch3.sort_values('データ内連番')
df_train_pitch3

C:\Users\bono_\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\bono_\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,データ内連番,球種,投球位置区域,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,右翼手ID,成績対象投手ID,成績対象打者ID,count,pitch_team_score
0,0,0,0,2017033101,1,1,2017-03-31,18:03:00,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1500001,1,右,先発,1,1,1,1,900410,4,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,900410,0-0-0,0
1,1,0,8,2017033101,2,2,2017-03-31,18:03:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,2,1500001,1,右,先発,1,1,2,2,900410,4,左,1,中,1,0,0,0,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,900410,1-0-0,0
2,2,0,5,2017033101,3,3,2017-03-31,18:03:23,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,3,1500001,1,右,先発,1,1,3,3,900410,4,左,1,中,1,0,0,0,0,2,___,NaN,NaN,NaN,1300027,900397,1500001,900410,2-0-0,0
3,3,0,12,2017033101,4,4,2017-03-31,18:04:01,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1500001,1,右,先発,1,2,4,4,11436,4,右,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,11436,0-0-1,0
4,4,0,8,2017033101,5,5,2017-03-31,18:04:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1500001,1,右,先発,1,2,5,5,11436,4,右,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,11436,1-0-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257112,257112,0,12,2017101002,293,288,2017-10-10,20:55:30,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,1,1600115,5,右,救援,6,7,20,13,1400160,4,右,7,捕,1,6,1,2,0,0,1__,700035.0,NaN,NaN,500066,1400170,1600115,1400160,0-0-2,-5
257113,257113,0,10,2017101002,294,289,2017-10-10,20:55:53,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,2,1600115,5,右,救援,6,7,21,14,1400160,4,右,7,捕,1,6,1,2,0,1,1__,700035.0,NaN,NaN,500066,1400170,1600115,1400160,1-0-2,-5
257114,257114,0,1,2017101002,295,290,2017-10-10,20:56:11,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,3,1600115,5,右,救援,6,7,22,15,1400160,4,右,7,捕,1,6,1,2,1,1,1__,700035.0,NaN,NaN,500066,1400170,1600115,1400160,1-1-2,-5
257115,257115,0,9,2017101002,296,291,2017-10-10,20:56:39,5,4,5,甲子園,セ・リーグ公式戦,9,表,4,4,1600115,5,右,救援,6,7,23,16,1400160,4,右,7,捕,1,6,1,2,1,2,1__,700035.0,NaN,NaN,500066,1400170,1600115,1400160,2-1-2,-5


In [42]:
df_date =df_train_pitch3['日付'].str.split('-', expand=True)
df_date

,0,1,2
0,2017,03,31
1,2017,03,31
2,2017,03,31
3,2017,03,31
4,2017,03,31
...,...,...,...
257112,2017,10,10
257113,2017,10,10
257114,2017,10,10
257115,2017,10,10


In [43]:
df_date.rename(columns={0: 'year', 1: 'month',2:'day'}, inplace=True)
df_date = df_date.loc[:,['month']]
df_date = df_date.astype({'month':int})
df_date.head()

,month
0,3
1,3
2,3
3,3
4,3


In [44]:
df_train_pitch3 = pd.concat([df_train_pitch3,df_date],axis=1)

In [45]:
df_train_pitch3.head(100)

,データ内連番,球種,投球位置区域,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,右翼手ID,成績対象投手ID,成績対象打者ID,count,pitch_team_score,month
0,0,0,0,2017033101,1,1,2017-03-31,18:03:00,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,1,1500001,1,右,先発,1,1,1,1,900410,4,左,1,中,1,0,0,0,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,900410,0-0-0,0,3
1,1,0,8,2017033101,2,2,2017-03-31,18:03:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,2,1500001,1,右,先発,1,1,2,2,900410,4,左,1,中,1,0,0,0,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,900410,1-0-0,0,3
2,2,0,5,2017033101,3,3,2017-03-31,18:03:23,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,1,3,1500001,1,右,先発,1,1,3,3,900410,4,左,1,中,1,0,0,0,0,2,___,NaN,NaN,NaN,1300027,900397,1500001,900410,2-0-0,0,3
3,3,0,12,2017033101,4,4,2017-03-31,18:04:01,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,1,1500001,1,右,先発,1,2,4,4,11436,4,右,2,二,1,0,0,1,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,11436,0-0-1,0,3
4,4,0,8,2017033101,5,5,2017-03-31,18:04:12,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,2,1500001,1,右,先発,1,2,5,5,11436,4,右,2,二,1,0,0,1,0,1,___,NaN,NaN,NaN,1300027,900397,1500001,11436,1-0-1,0,3
5,5,1,9,2017033101,6,6,2017-03-31,18:04:26,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,3,1500001,1,右,先発,1,2,6,6,11436,4,右,2,二,1,0,0,1,0,2,___,NaN,NaN,NaN,1300027,900397,1500001,11436,2-0-1,0,3
6,6,2,12,2017033101,7,7,2017-03-31,18:04:41,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,2,4,1500001,1,右,先発,1,2,7,7,11436,4,右,2,二,1,0,0,1,1,2,___,NaN,NaN,NaN,1300027,900397,1500001,11436,2-1-1,0,3
7,7,3,11,2017033101,8,8,2017-03-31,18:05:18,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,3,1,1500001,1,右,先発,1,3,8,8,1600184,4,右,3,三,1,0,0,2,0,0,___,NaN,NaN,NaN,1300027,900397,1500001,1600184,0-0-2,0,3
8,8,0,11,2017033101,9,9,2017-03-31,18:05:31,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,3,2,1500001,1,右,先発,1,3,9,9,1600184,4,右,3,三,1,0,0,2,1,0,___,NaN,NaN,NaN,1300027,900397,1500001,1600184,0-1-2,0,3
9,9,2,8,2017033101,10,10,2017-03-31,18:05:51,1,4,1,東京ドーム,セ・リーグ公式戦,1,表,3,3,1500001,1,右,先発,1,3,10,10,1600184,4,右,3,三,1,0,0,2,1,1,___,NaN,NaN,NaN,1300027,900397,1500001,1600184,1-1-2,0,3


In [46]:
df_train_pitch3.dtypes

データ内連番                int64
球種                    int64
投球位置区域                int64
試合ID                  int64
試合内連番                 int64
試合内投球数                int64
日付                   object
時刻                   object
ホームチームID              int64
アウェイチームID             int64
球場ID                  int64
球場名                  object
試合種別詳細               object
イニング                  int64
表裏                   object
イニング内打席数              int64
打席内投球数                int64
投手ID                  int64
投手チームID               int64
投手投球左右               object
投手役割                 object
投手登板順                 int64
投手試合内対戦打者数            int64
投手試合内投球数              int64
投手イニング内投球数            int64
打者ID                  int64
打者チームID               int64
打者打席左右               object
打者打順                  int64
打者守備位置               object
打者試合内打席数              int64
プレイ前ホームチーム得点数         int64
プレイ前アウェイチーム得点数        int64
プレイ前アウト数              int64
プレイ前ボール数              int64
プレイ前ストライク数          

In [47]:
pitch_group = df_train_pitch3.groupby(['試合ID','イニング','表裏','イニング内打席数'])

In [48]:
lag_pitch = pd.DataFrame(pitch_group['球種'].shift(1))

In [49]:
lag_pitch = lag_pitch.fillna(15)
lag_pitch = lag_pitch.rename(columns={'球種': 'lag_pitch'})
lag_pitch['lag_pitch'] = lag_pitch['lag_pitch']
lag_pitch = lag_pitch.astype('int64')
lag_pitch

,lag_pitch
0,15
1,0
2,0
3,15
4,0
...,...
257112,15
257113,0
257114,0
257115,0


In [50]:
df_train_pitch4 = pd.concat([df_train_pitch3,lag_pitch],axis=1)

In [57]:
pd.to_pickle(df_train_pitch4,'./df_train_pitch4.pkl')

In [53]:
df_train_pitch4 = pd.read_pickle('./df_train_pitch4.pkl')

In [54]:
df_train_pitch4.to_csv('./df_train_pitch4.csv')

# playerファイルの加工

In [58]:
df_train_player

,年度,チームID,チーム名,選手ID,選手名,育成選手F,背番号,位置,投,打,身長,体重,生年月日,出身高校ID,出身高校名,出身大学ID,出身大学名,社会人,ドラフト年,ドラフト種別,ドラフト順位,年俸,出身国,出身地,血液型
0,2017,1,巨人,11343,相川 亮二,0,23,捕手,右,右,183,86,1976/7/11,30227,東京学館,0,不明,NaN,1994.0,NaN,5.0,4500,日本,千葉,A
1,2017,1,巨人,11726,實松 一成,0,27,捕手,右,右,177,85,1981/1/18,674,佐賀学園,0,不明,NaN,1998.0,NaN,1.0,3700,日本,佐賀,A
2,2017,1,巨人,12049,阿部 慎之助,0,10,内野手,右,左,180,97,1979/3/20,1114,安田学園,895,中央大,NaN,2000.0,NaN,1.0,26000,日本,千葉,A
3,2017,1,巨人,12107,杉内 俊哉,0,18,投手,左,左,175,82,1980/10/30,740,鹿児島実,0,不明,三菱重工長崎,2001.0,NaN,3.0,5000,日本,福岡,A
4,2017,1,巨人,12179,大竹 寛,0,17,投手,右,右,184,94,1983/5/21,179,浦和学院,0,不明,NaN,2001.0,NaN,1.0,7000,日本,埼玉,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,2017,376,楽天,1600102,千葉 耕太,1,128,投手,右,右,191,87,1998/4/22,67,花巻東,0,不明,NaN,2016.0,育成選手,1.0,240,日本,岩手,A
907,2017,376,楽天,1600103,南 要輔,1,129,内野手,右,右,175,73,1994/8/7,272,東海大菅生,910,明星大,NaN,2016.0,育成選手,2.0,250,日本,東京,O
908,2017,376,楽天,1600104,向谷 拓巳,1,130,内野手,右,左,170,68,1997/2/24,30303,大和広陵,0,不明,ＢＦＬ・兵庫,2016.0,育成選手,3.0,250,日本,奈良,O
909,2017,376,楽天,1600105,木村 敏靖,1,131,投手,右,左,176,78,1995/9/14,32654,御所実,0,不明,履正社医療スポーツ専門学校,2016.0,育成選手,4.0,250,日本,奈良,O


In [59]:
df_train_player2 = pd.DataFrame(df_train_player)

In [60]:
df_train_player['年齢'] = df_train_player['年度']-df_train_player['ドラフト年']

In [61]:
df_age = df_train_player['生年月日'].str.split('/',expand = True)
df_age.head()

,0,1,2
0,1976,7,11
1,1981,1,18
2,1979,3,20
3,1980,10,30
4,1983,5,21


In [62]:
df_age.rename(columns={0:'birth'},inplace=True)
df_age =df_age.drop([1,2],axis=1)
df_age

,birth
0,1976
1,1981
2,1979
3,1980
4,1983
...,...
906,1998
907,1994
908,1997
909,1995


In [63]:
df_age.isnull().sum()

birth    0
dtype: int64

In [64]:
df_age = df_age.astype('int64')

In [65]:
df_train_player3 = pd.concat([df_train_player2,df_age],axis=1)

In [66]:
df_train_player3['age'] = df_train_player3['年度']-df_train_player3['birth']

In [67]:
df_train_player4 =df_train_player3.loc[:,['年度','選手ID','age']]

In [68]:
df_merge = pd.merge(df_train_pitch4,df_train_player4,left_on='投手ID',right_on='選手ID',how='left')

In [69]:
df_merge2 = df_merge.drop(['投球位置区域','試合ID','時刻','ホームチームID','アウェイチームID','球場ID','球場名','試合種別詳細','イニング内打席数','投手ID','投手チームID','打者ID','打者チームID','プレイ前ホームチーム得点数','プレイ前アウェイチーム得点数','一塁走者ID','二塁走者ID','三塁走者ID','捕手ID','成績対象投手ID','成績対象打者ID','年度','選手ID'],axis=1)

In [70]:
df_merge2

,データ内連番,球種,試合内連番,試合内投球数,日付,イニング,表裏,打席内投球数,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,右翼手ID,count,pitch_team_score,month,lag_pitch,age
0,0,0,1,1,2017-03-31,1,表,1,右,先発,1,1,1,1,左,1,中,1,0,0,0,___,900397,0-0-0,0,3,15,29
1,1,0,2,2,2017-03-31,1,表,2,右,先発,1,1,2,2,左,1,中,1,0,0,1,___,900397,1-0-0,0,3,0,29
2,2,0,3,3,2017-03-31,1,表,3,右,先発,1,1,3,3,左,1,中,1,0,0,2,___,900397,2-0-0,0,3,0,29
3,3,0,4,4,2017-03-31,1,表,1,右,先発,1,2,4,4,右,2,二,1,1,0,0,___,900397,0-0-1,0,3,15,29
4,4,0,5,5,2017-03-31,1,表,2,右,先発,1,2,5,5,右,2,二,1,1,0,1,___,900397,1-0-1,0,3,0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257112,257112,0,293,288,2017-10-10,9,表,1,右,救援,6,7,20,13,右,7,捕,1,2,0,0,1__,1400170,0-0-2,-5,10,15,19
257113,257113,0,294,289,2017-10-10,9,表,2,右,救援,6,7,21,14,右,7,捕,1,2,0,1,1__,1400170,1-0-2,-5,10,0,19
257114,257114,0,295,290,2017-10-10,9,表,3,右,救援,6,7,22,15,右,7,捕,1,2,1,1,1__,1400170,1-1-2,-5,10,0,19
257115,257115,0,296,291,2017-10-10,9,表,4,右,救援,6,7,23,16,右,7,捕,1,2,1,2,1__,1400170,2-1-2,-5,10,0,19


In [71]:
pd.to_pickle(df_merge2,'df_merge2.pkl')